# Исследование данных об аренде самокатов

## Введение

**Краткое описание проекта:** В нашем распоряжении данные о некоторых пользователях из нескольких городов, а также об их поездках. Необходимо проанализировать данные и проверить несколько гипотез, которые могут помочь бизнесу вырасти.

**Цель проекта:** Проверить статистические гипотезы, а как следствие получить информацию, которая может помочь сервису аренды самокатов стать лучше и вырасти как бизнес.

**Описание данных:** В основных данных есть информация о пользователях, их поездках и подписках (3 датасета).

1. **Пользователи — `users_go.csv`**

- `user_id` - уникальный идентификатор пользователя

- `name` - имя пользователя

- `age` - возраст

- `city` - город

- `subscription_type` - тип подписки (free, ultra)


2. **Поездки — `rides_go.csv`**

- `user_id` - уникальный идентификатор пользователя

- `distance` - расстояние, которое пользователь проехал в текущей сессии (в метрах)

- `duration` - продолжительность сессии (в минутах) — время с того момента, как пользователь нажал кнопку «Начать поездку» до момента, как он нажал кнопку «Завершить поездку»

- `date` - дата совершения поездки

3. **Подписки — `subscriptions_go.csv`**

- `subscription_type` - тип подписки

- `minute_price` - стоимость одной минуты поездки по данной подписке

- `start_ride_price` - стоимость начала поездки

- `subscription_fee` - стоимость ежемесячного платежа

**План работ:**
- Изучить данные
- Провести предобработку данных: обработать пропуски, устранить дубликаты, обработать типы данных
- Провести исследовательский анализ данных (описать и визуализируйте общую информацию о пользователях и поездках в следующих разрезах: частота встречаемости городов; соотношение пользователей с подпиской и без подписки; возраст пользователей; расстояние, которое пользователь преодолел за одну поездку; продолжительность поездок).
- Объединить данные в один датафрейм
- Рассчитать выручку
- Проверить гипотезы
- Построить распределения

## Загрузка и первоначальное изучение данных

In [1]:
# Импорты
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt
import numpy as np
import warnings

In [2]:
# Загрузка с Google Drive
users_df = pd.read_csv('https://drive.google.com/uc?export=download&id=1QzCHinXigi14LxXMoYwH1i4QYxzECYUD', sep=',')
rides_df = pd.read_csv('https://drive.google.com/uc?export=download&id=11rEIIP6AXCXD5F6GVTLbHiDn0oR1GCij', sep=',')
subscriptions_df = pd.read_csv('https://drive.google.com/uc?export=download&id=1DG_L3qx3AFq1PDh6GWuspi5goa9yomwr', sep=',')

In [3]:
# Опционально: убирает все предупреждения
warnings.filterwarnings("ignore")

Выведем на экран первые 10 строк датафрейма `users_df`

In [4]:
users_df.head(10)

Изучим также общую информацию о данных

In [5]:
users_df.info()

В данных отсутствуют пропуски, типы данных приведены к оптимальным.

Выведем на экран первые 10 строк датафрейма `rides_df`

In [6]:
rides_df.head(10)

Изучим также общую информацию о данных

In [7]:
rides_df.info()

В данных отсутствуют пропуски, но тип данных столбца `'date'` необходимо будет перевести в тип `datetime`, это одна из будущих подзадач следующего раздела - Предобработка данных.

Перейдем к последнему датасету - `'subscriptions_df'`. Выведем весь датафрейм на экран (в нем всего 2 строки)

In [8]:
subscriptions_df.head()

Выведем общую информацию

In [9]:
subscriptions_df.info()

Типы данных оптимальны, пропуски отсутствуют.

## Предобработка данных

В наших данных отсутствуют пропуски, поэтому начать предобработку данных можно с преобразования типов данных

### Обработка типов данных

Приведем столбец `date` датафрейма `rides_df` в формат `datetime` pandas.

In [10]:
rides_df['date'] = pd.to_datetime(rides_df['date'], format='%Y-%m-%d')
rides_df.info()

Тип данных успешно изменен на необходимый нам. 

### Добавление вспомогательных столбцов

Добавим столбец `month`, который будет хранить информацию о месяце, в котором состоялась поездка.

In [11]:
rides_df['month'] = pd.DatetimeIndex(rides_df['date']).month
rides_df.head()

Столбец добавлен. Проверим, данные за какой период у нас в распоряжении

In [12]:
print('Начальная дата:', rides_df['date'].unique().min(), '\nКонечная дата:', rides_df['date'].unique().max())

В нашем распоряжении данные за 2021 год.

### Обработка пропущенных значений и дубликатов

В ходе изучения данных (Раздел 1), был получен вывод об **отсутствии пропусков в данных**. Перейдем к устранению дубликатов.

#### Обработка дубликатов `users_df`
Начнем с `users_df`: проверим, могли ли в данных возникнуть неявные дубликаты: посмотрим на уникальные значения столбцов `'name'`

In [13]:
print(sorted(users_df['name'].unique()))

Обнаружены неявные дубликаты: неполная форма имени (прим. Влад, Владислав). Обработаем такие дубликаты

In [14]:
users_df.loc[users_df['name'] == 'Влад', 'name'] = 'Владислав'
users_df.loc[users_df['name'] == 'Влада', 'name'] = 'Владислава'
print(sorted(users_df['name'].unique()))

Теперь обратимся к столбцу `'city'` и выведем его уникальные значения

In [15]:
sorted(users_df['city'].unique())

В столбце  `'city'` неявные дубликаты отсутствуют.

Неявные дубликаты устранены.

In [16]:
print(f'Количество явных дубликатов users_df: {users_df.duplicated().sum()}')

Удалим явные дубликаты

In [17]:
users_df = users_df.drop_duplicates()
print(f'Количество явных дубликатов users_df: {users_df.duplicated().sum()}')

Все дубликаты `users_df` устранены.

#### Обработка дубликатов `rides_df`

В данных не может быть неявных дубликатов, поэтому перейдем сразу к подсчету явных дубликатов

In [18]:
print(f'Количество явных дубликатов rides_df: {rides_df.duplicated().sum()}')

Дубликаты отсутствуют

#### Обработка дубликатов `subscriptions_df`

В датафрейме `subscriptions_df` всего 2 строки, и они не являются дубликатами, поэтому не имеет смысла проводить доплнительные проверки.

**Вывод:** Все дубликаты в данных устранены.

**Общий вывод по Предобработке данных:**
- Дубликаты в данных отсутствуют
- Тип данных колонки с датой изменен на `datetime` pandas
- Добавлен вспомогательный столбец `'month'` для дальнейшей группировки
- Обработаны неявные дубликаты
- Все дубликаты устранены

## Исследовательский анализ данных

В текущем разделе будет описана и визуализирована общая информация о пользователях и поездках:

**3.1** частота встречаемости городов;

**3.2** соотношение пользователей с подпиской и без подписки;

**3.3** возраст пользователей;

**3.4** расстояние, которое пользователь преодолел за одну поездку;

**3.5** продолжительность поездок.

### Исследование частоты встречаемости городов

Сгруппируем данные о поездках по городам, результат сохраним в виде сводной таблицы, выведем ее на экран

In [19]:
city_frequency_pivot = (users_df.pivot_table(index='city', values='user_id', aggfunc='count')
                        .rename(columns={'user_id': 'amount'})
                        .sort_values(by='amount', ascending=False)
                        .reset_index()) 
city_frequency_pivot

Визуализируем полученные данные с помощью столбчатой диаграммы

In [20]:
city_frequency_pivot.sort_values(by='amount', ascending=False).plot(kind='barh', y='amount', x='city', legend=False)
plt.title('Частота встречаемости городов')
plt.xlabel('Количество пользователей')
plt.ylabel('Город')
plt.show()

**Вывод:** 
- Наибольшее количество пользователей - из Пятигорска, их 219. Далее следует Екатеринбург с показателем 204 пользователя.
- Наименьшее количество пользователей - из Москвы, их 168.

### Исследование соотношения пользователей с подпиской и без подписки

Сгруппируем данные датафрейма о пользователях по типам подписки и найдем количество пользователей с подписками `ultra` и `free`, а также их долю от общего количества. Результат сохраним в сводную таблицу. Визуализируем данные с помощью круговой диаграммы.

In [21]:
subscription_pivot = (users_df.pivot_table(index='subscription_type', values='user_id', aggfunc='count')
                      .reset_index()
                      .rename(columns={'user_id': 'amount'}))
subscription_pivot['share'] = round((subscription_pivot['amount'] / subscription_pivot['amount'].sum()), 2)
subscription_pivot

In [22]:
subscription_pivot.plot(kind='pie', y='amount', labels=['без подписки', 'с подпиской'], legend=False, autopct='%1.0f%%')
plt.title('Доли пользователей с подпиской и без')
plt.ylabel('')
plt.show()

**Вывод:** 835 пользователей (54%) не пользуются платной подпиской, а 699 (46%) пользуются.

### Исследование возраста пользователей

Сгруппируем данные датафрейма о пользователях по возрасту и найдем их долю от общего количества. Результат сохраним в сводную таблицу. Визуализируем данные с помощью столбчатой диаграммы.

In [23]:
age_pivot = (users_df.pivot_table(index='age', values='user_id', aggfunc='count')
                      .reset_index()
                      .rename(columns={'user_id': 'amount'}))
age_pivot['share'] = round(age_pivot['amount'] / age_pivot['amount'].sum(), 2)
age_pivot.plot(kind='bar', y='amount', x='age', grid=True, figsize=(10, 5), legend=False)
plt.title('Распределение возрастов пользователей')
plt.xlabel('Возраст')
plt.xticks(rotation=0)
plt.show()

Выведем 10 строк с данными о самых частых возрастах

In [24]:
age_pivot.sort_values(by='amount', ascending=False).head(10)

Выведем общее описание полученных данных

In [25]:
age_pivot['age'].describe()

**Вывод**:
- Распредедение возрастов пользователей близко к нормальному распределению со **средним 26 лет**, **стандартным отклонением 8.7**.
- Аномалии в данных отсутствуют.

### Исследование расстояния, пройденного в ходе поездок

Изучим данные о расстоянии, которое проезжают пользователи во время своих поездок. Для этого нам понадобится построить гистограмму столбца `'distance'` датафрейма `rides_df`, так как расстояние - непрерывная величина

In [26]:
rides_df.plot(kind='hist', y='distance', bins=30, legend=False)
plt.title('Расстояние, пройденное в поездках')
plt.xlabel('Расстояние в метрах')
plt.ylabel('Количество поездок')
plt.show()

Выведем также общее описание данных

In [27]:
rides_df['distance'].describe()

**Вывод:**

- Пользователи сервиса в среднем проезжают **около 3070 метров**, это значение близко и к **медиане (3133 метра)** и к **среднему значению по выборке (3070 метров)**
- Обычно пользователь проезжают 3776 метров или меньше (в 75% случаев)
- Необычно долгие поездки - более 6500 метров, но они не выглядят аномально и не противоречат общей картине
- На гистограмме присутствует локальный пик в районе 500 метров, его существование может быть связано с тем, что у пользователей могут быть разные цели поездки: *доехать из точки А в точку В* или *покататься на маленькой дистанции*

### Исследование длительности поездок

Изучим данные о расстоянии, которое проезжают пользователи во время своих поездок. Для этого нам понадобится построить гистограмму столбца `'duration'` датафрейма `rides_df`, так как длительность поездок - непрерывная величина

In [28]:
rides_df.plot(kind='hist', bins=50, y='duration', grid=True, legend=False)
plt.title('Длительность поездок')
plt.xlabel('Длительность поездки в минутах')
plt.ylabel('Количество поездок')
plt.show()

Распределение напоминает нормальное, но есть небольшой пик в районе 1 минуты. Изучим его более детально

In [29]:
rides_df.loc[rides_df['duration'] <= 1]

За 30 секунд проехать 4 и более километра на самокате - физически невозможно, поэтому считаем данные аномальными. Найдем их долю среди всех данных

In [30]:
print(f'Доля аномальных значений {(len(rides_df.loc[rides_df["duration"] <= 1]) / len(rides_df["duration"])):.2%}')

Аномалии занимают очень маленький процент от данных, отбросим их, чтобы не учитывать в результатах исследования

In [31]:
rides_df = rides_df.loc[rides_df["duration"] > 1]
print(f'Доля аномальных значений {(len(rides_df.loc[rides_df["duration"] <= 1]) / len(rides_df["duration"])):.2%}')

In [32]:
rides_df.plot(kind='hist', bins=50, y='duration', grid=True, legend=False)
plt.title('Длительность поездок')
plt.xlabel('Длительность поездок в минутах')
plt.ylabel('Количество поездок')
plt.show()

Аномалия устранена, теперь распределение ближе к нормальному.

In [33]:
rides_df['duration'].describe()

**Вывод:** 
- Распредедение длительности поездок близко к нормальному распределению со **средним 17 минут**, **стандартным отклонением почти 6 минут**. Медиана составляет также 17 минут.
- Аномалии в данных отсутствуют.

## Объединение данных и визуализация

### Объединение данных
Объединим данные о пользователях, поездках и подписках в один датафрейм. Для начала выведем количество строк датафрейма, к которому будем присоединять колонки

In [34]:
print(rides_df.shape[0])

Теперь объединим датафреймы и выведем первые 5 строк получившегося датафрейма

In [35]:
data_general = rides_df.merge(users_df, how='left', on='user_id')
data_general = data_general.merge(subscriptions_df, how='left', on='subscription_type')
data_general.head()

Теперь выведем общую информацию о получившемся датафрейме, чтобы убедиться в отсутствии пропусков и в том, что не добавлено лишних строк

In [36]:
data_general.info()

Пропусков, лишних строк не появилось. Можно двигаться дальше.

### Разделение данных о пользователях с подпиской и без
Для дальнейшего удобства, создадим два датафрейма - `ultra_df` и `free_df`. Они будут хранить в себе данные о поездках с платной подпиской и без нее соответственно.

Создадим датафрейм `ultra_df` и выведем на экран первые 5 строк 

In [37]:
ultra_df = data_general.loc[data_general['subscription_type'] == 'ultra'].reset_index(drop=True)
ultra_df.head()

Создадим датафрейм `free_df` и выведем на экран первые 5 строк 

In [38]:
free_df = data_general.loc[data_general['subscription_type'] == 'free'].reset_index(drop=True)
free_df.head()

### Визуализация информации о расстоянии и времени поездок с подпиской и без 

Для визуализации данных будем использовать гистограммы, как и ранее, так как расстояние и время - непрерывные величины.

Сначала проанализируем поездки с подпиской и без независимо, а далее проведем сравнение.

#### Поездки с подпиской
Рассмотрим сначала датафрейм `ultra_df`, он содержит информацию о поездках с подпиской. Построим гистограмму расстояния и выведем общее описание данных

In [39]:
ultra_df.plot(kind='hist', y='distance', bins=30, legend=False)
plt.title('Распределение расстояния, пройденного в поездках среди пользоваталей с подпиской')
plt.xlabel('Расстояние, пройденное в поездке в метрах')
plt.ylabel('Количество поездок')
plt.show()
ultra_df['distance'].describe()

**Вывод:**
- Количество поездок - 6500
- Среднее значение - **3115 метров**, медиана - **3148 метров**. 
- В 75% поездок было пройдено **3560 метров** или меньше.
- На графике присутствует локальный пик в районе 1000 метров, но на аномалию этот пик не похож.

Теперь перейдем к **времени в поездке**. Построим для него гистограмму и выведем общую информацию

In [40]:
ultra_df.plot(kind='hist', y='duration', bins=30, legend=False)
plt.title('Распределение длительности поездки среди пользоваталей с подпиской')
plt.xlabel('Длительность поездки в минутах')
plt.ylabel('Количество поездок')
plt.show()
ultra_df['duration'].describe()

**Вывод:**
- Распределение времени в поездке близко к нормальному, со **средним 18.5 минут**, стандартное отклонение **5.5 минут**. Медианное значение **18 минут**.
- Более 75% поездок заканчиваются за 22 минуты или меньше

#### Поездки без подписки
Теперь рассмотрим датафрейм `free_df`, он содержит информацию о поездках без подписки. Построим гистограмму расстояния и выведем общее описание данных

In [41]:
free_df.plot(kind='hist', y='distance', bins=30, legend=False)
plt.title('Распределение расстояния, пройденного в поездках среди пользоваталей без подписки')
plt.xlabel('Расстояние, пройденное в поездке в метрах')
plt.ylabel('Количество поездок')
plt.show()
free_df['distance'].describe()

**Вывод:**
- Количество поездок - 11473
- Среднее значение - **3028 метров**, медиана - **3102 метров**. 
- В 75% поездок было пройдено **3883 метра** или меньше.
- На графике присутствует локальный пик в районе 500 метров, но на аномалию этот пик не похож.

Теперь перейдем к **времени в поездке**. Построим для него гистограмму и выведем общую информацию

In [42]:
free_df.plot(kind='hist', y='duration', bins=30, legend=False)
plt.title('Длительность поездок среди пользователей без подписки')
plt.xlabel('Длительность поездки в минутах')
plt.ylabel('Количество поездок')
plt.show()
free_df['duration'].describe()

**Вывод:**
- Распределение близко к нормальному со **средним 17.5 минут**, стандартным отклонением 6.1, **медианой 17.4**
- 75% поездок длились не более 21 минуты

#### Сравнение (общий вывод к подразделу 4.3)

Для наглядности наложим распределения на один график

In [43]:
# Расстояние
plt.figure(figsize=(15, 7))
free_df['distance'].hist(bins=30, 
             color='#FFD89C', alpha=0.7, label='Пользователи без подписки')
ultra_df['distance'].hist(bins=30,
              color='#A0C49D', alpha=0.7, label='Пользователи с подпиской')
plt.vlines(x=free_df['distance'].mean(), ymin=0, ymax=1001, color='y')
plt.vlines(x=ultra_df['distance'].mean(), ymin=0, ymax=1001, color='g')
plt.title('Распределение расстояния,\n пройденного в поездках среди пользоваталей с подпиской')
plt.xlabel('Расстояние, пройденное в поездке в метрах')
plt.ylabel('Количество поездок')
plt.legend()
plt.show()

# Длительность
plt.figure(figsize=(15, 7))
free_df['duration'].hist(bins=30, 
             color='#FFD89C', alpha=0.7, label='Пользователи без подписки')
ultra_df['duration'].hist(bins=30,
              color='#A0C49D', alpha=0.7, label='Пользователи с подпиской')
plt.vlines(x=free_df['duration'].mean(), ymin=0, ymax=1001, color='y')
plt.vlines(x=ultra_df['duration'].mean(), ymin=0, ymax=1001, color='g')
plt.title('Распределение длительности поездки\n среди пользоваталей с подпиской')
plt.xlabel('Длительность поездки в минутах')
plt.ylabel('Количество поездок')
plt.legend()
plt.show()

- Поездок без подписок почти в 2 раза больше (6500 с подпиской и 115473 без)
- **Среднее расстояние** у поездок с подписками немного выше *(3115 против 3028 метров)*, это может быть связано с тем, что среди поездок без подписки есть некоторое количество поездок с расстоянием **менее 500** метров. Медианы почти не отличаются. В 75% поездок с подписками было пройдено **3560 метров** или меньше, а в 75% поездок без подписки расстояние было **3883**, это значительная разница
- **Средняя длительность** поездок с подписками почти на 1 минуту больше, чем без подписок *(18.5 против 17.5)*, медианная на 0.7 минуты больше *(18.1 против 17.4)*. 75% поездок с подпиской завершены за 22 минуты или меньше, а без подписки - 21.6 минут или меньше, разница не так значительна.

## Подсчет выручки


Составим таблицу, в которой вычислим суммарное расстояние, количество поездок и суммарное время для каждого пользователя за каждый месяц. До этого предварительно округлим до ближайшего большего целого числа значения длительности поездок в исходном датафрейме.

Также добавим вспомогателный столбец с типом подписки, а также из датафрейма с данными о подписках добавим в получившуюся таблицу информацию о стоимости минуты, старта и подписки *(это нужно для подсчета выручки)*. Выведем первые 5 строк 

In [44]:
# Округлим длительность поездок
data_general['duration'] = np.ceil(data_general['duration'])

revenue_pivot = (data_general
                 .pivot_table(index=['user_id', 'month'],
                              values=['distance', 'duration'],
                              aggfunc=['sum', 'count']).reset_index())
revenue_pivot.columns = ['user_id', 'month', 'distance', 'duration', 'amount', 'subscription_type']

# Добавим столбец с типом подписки
revenue_pivot.loc[revenue_pivot['user_id'].isin(ultra_df['user_id']), 'subscription_type'] = 'ultra'
revenue_pivot.loc[revenue_pivot['user_id'].isin(free_df['user_id']), 'subscription_type'] = 'free'
revenue_pivot = revenue_pivot.merge(subscriptions_df, how='left', on='subscription_type')

revenue_pivot.head()

Добавим в получившуюся таблицу столбец `'revenue'`, который будет хранить информацию о суммарной выручке за месяц. Выведем на экран первые 5 строк

In [45]:
revenue_pivot['revenue'] = (revenue_pivot['duration'] * revenue_pivot['minute_price'] 
                            + revenue_pivot['start_ride_price'] * revenue_pivot['amount'] 
                            + revenue_pivot['subscription_fee'])
revenue_pivot.head()

## Статистический анализ данных

В текущем разделе будет проведен статистический анализ данных, чтобы получить ответы на вопросы:

6.1 Тратят ли пользователи с подпиской больше времени на поездки? 

6.2 Превышает ли среднее расстояние, которое проезжают пользователи с подпиской за одну поездку 3130 метров?

6.3 Будет ли помесячная выручка от пользователей с подпиской по месяцам выше, чем выручка от пользователей без подписки?

6.4 *Дополнительный вопрос:* Представим такую ситуацию: техническая команда сервиса обновила сервера, с которыми взаимодействует мобильное приложение. Она надеется, что из-за этого количество обращений в техподдержку значимо снизилось. Некоторый файл содержит для каждого пользователя данные о количестве обращений до обновления и после него. Какой тест вам понадобился бы для проверки этой гипотезы?

### Тратят ли пользователи с подпиской больше времени на поездки?

Для ответа на вопрос, сформулируем нулевую и альтернативную гипотезы для дальнейшей проверки:
- **Нулевая гипотеза (H0)**: Пользователи с подпиской тратят столько же времени на поездки, как и пользователи без подписки
- **Альтернативная гипотеза (H1)**: Пользователи с подпиской тратят больше времени на поездки

Для того, чтобы проверить выдвинутые гипотезы, проведем двухвыборочный правосторонний Т-тест для независимых выборок. Уровень статистической значимости установим на уровне 0.01.

In [46]:
alpha = 0.01
duration_comparement = st.ttest_ind(ultra_df['duration'], free_df['duration'], alternative='greater')

print(f'Значение p-value: {duration_comparement.pvalue}')
if duration_comparement.pvalue < alpha:
    print('Имеем достаточные основания отклонить H0')
else:
    print('Не имеем достаточных оснований отклонить H0 в пользу H1')

**Вывод:** Статистически доказано, что есть достаточные основания отклонить гипотезу о равенстве средних расстояний в пользу альтернативной правосторонней гипотезы, что среднее расстояние у пользователей с подпиской больше, чем у пользователей без подписки.

### Превышает ли среднее расстояние, которое проезжают пользователи с подпиской за одну поездку 3130 метров?

Для ответа на вопрос, сформулируем нулевую и альтернативную гипотезы:
- **Нулевая гипотеза (H0)**: Среднее расстояние, которое проезжают пользователи с подпиской за одну поездку равно 3130 метров.
- **Альтернативная гипотеза (H1)**: Среднее расстояние, которое проезжают пользователи с подпиской за одну поездку, больше 3130 метров.

Для того, чтобы проверить выдвинутые гипотезы, проведем одновыборочный правосторонний Т-тест. Уровень статистической значимости установим на уровне 0.01

In [47]:
h0_distance = 3130
distance_test = st.ttest_1samp(ultra_df['distance'], h0_distance, alternative='greater')

print(f'P-value: {distance_test.pvalue}')
if distance_test.pvalue < alpha:
    print('Имеем достаточные основания отклонить H0')
else:
    print('Не имеем достаточных оснований отклонить H0 в пользу H1')

Так как мы получили p-value равное почти 0.92, мы не можем отвергнуть нулевую гипотезу о равенстве в пользу H1.

**Вывод:**
- Среднее расстояние, которое проезжают пользователи с подпиской за одну поездку, не превышает 3130 метров.
- Износ самокатов оптимален.

### Будет ли помесячная выручка от пользователей с подпиской по месяцам выше, чем выручка от пользователей без подписки?

Для ответа на вопрос, сформулируем нулевую и альтернативную гипотезы:
- **Нулевая гипотеза (H0)**: Средняя выручка по месяцам от пользователей с подпиской равна средней выручке по месяцам от пользователей без подписки
- **Альтернативная гипотеза (H1)**: Средняя выручка по месяцам от пользователей с подпиской больше средней выручке по месяцам от пользователей без подписки

Для того, чтобы проверить выдвинутые гипотезы, проведем двухвыборочный правосторонний Т-тест для независимых выборок. Уровень статистической значимости установим на уровне 0.01

In [48]:
# Проверка гипотезы
revenue_test = (st.ttest_ind
                (revenue_pivot.loc[revenue_pivot['subscription_type'] == 'ultra', 'revenue'],
                revenue_pivot.loc[revenue_pivot['subscription_type'] == 'free', 'revenue'], 
                alternative='greater')
               )
print(f'P-value: {revenue_test.pvalue}')
if revenue_test.pvalue < alpha:
    print('Имеем основания отвергнуть H0')
else:
    print('Не имеем достаточно оснований отвергнуть H0 в пользу H1')

**Вывод:** 
- Статистически доказано, что есть основания отвергнуть гипотезу о равенстве выручки в пользу правосторонней альтернативной гипотезы (средняя помесячная выручка пользователей с подпиской выше, чем без подписки)
- Пользователи с подпиской приносят больше выручки, а значит они выгоднее

### Ответ на вопрос

Представим такую ситуацию: техническая команда сервиса обновила сервера, с которыми взаимодействует мобильное приложение. Она надеется, что из-за этого количество обращений в техподдержку значимо снизилось. Некоторый файл содержит для каждого пользователя данные о количестве обращений до обновления и после него. Какой тест нам понадобился бы для проверки этой гипотезы?

Сформулируем гипотезу и альтернативную:
- **Нулевая гипотеза (H0)**: Количество обращений в техподдержку после обновления серверов не изменилось
- **Альтернативная гипотеза (H1)**: Количество обращений в техподдержку после обновления серверов снизилось

Так как предполагается, что над выборкой будут произведены некоторые действия, и полученную в результате выборку необходимо будет сравнить с первоначальной, подойдет двухвыборочный правосторонний Т-тест для зависимых выборок. Он проводится с помощью метода `scipy.stats.ttest_rel(sample_before, sample_after, alternative='greater')`.

**Общий вывод к Проверке гипотез:**
- Так как статистически доказано, что есть достаточные основания отклонить гипотезу о равенстве средних расстояний в пользу альтернативной правосторонней гипотезы, что среднее расстояние у пользователей с подпиской больше, чем у пользователей без подписки, **пользователи с подпиской выгоднее, чем без подписки**
- Так как у нас недостаточно оснований отвергнуть гипотезу о равенстве среднего расстояния в поездке оптимальному значению 3130 метров, но достаточно оснований отклонить ее в пользу левосторонней гипотезы при значении 3140 метров, можно заключить, что износ самокатов не минимален.
- Так как статистически доказано, что есть основания отвергнуть гипотезу о равенстве выручки в пользу правосторонней альтернативной гипотезы (средняя помесячная выручка пользователей с подпиской выше, чем без подписки), пользователи с подпиской будут приносить больше денег

## Распределения

### Найти количество писем, при отправке которого вероятность получить 100 продлений около 5%

Отделу маркетинга GoFast поставили задачу: нужно провести акцию с раздачей промокодов на один бесплатный месяц подписки, в рамках которой как минимум 100 существующих клиентов должны продлить эту подписку. То есть по завершении периода действия подписки пользователь может либо отказаться от неё, либо продлить, совершив соответствующий платёж. 

Эта акция уже проводилась ранее и по итогу выяснилось, что после бесплатного пробного периода подписку продлевают 10% пользователей. Выясним, какое минимальное количество промокодов нужно разослать, чтобы вероятность не выполнить план была примерно 5%.

Для этого подберем соответствующий параметр `n` для биномиального распределения, а далее рассчитаем с помощью функции `cdf()` вероятность хвоста слева от 100 продлений, она должна быть около 0.05. Используем аппроксимацию нормальным распределением.

In [49]:
# Зададим распределение
success_prob = 0.1
n = 100
goal_amount = 100

goal_prob = 1

while round(goal_prob, 2) > 0.05:
    n += 1
    mu = n * success_prob
    sigma = np.sqrt(n*success_prob*(1-success_prob))
    renewal_distr = st.norm(mu, sigma)
    goal_prob = renewal_distr.cdf(100)

print(f'Вероятность левого хвоста от 100: {renewal_distr.cdf(goal_amount)} достигается при N равном {n}')

Построим график распределения

In [50]:
x = list(range(max(0, int(mu - 4 * sigma)), int(mu + 4 * sigma)))
plt.plot(x, st.binom.pmf(x, n, success_prob), 'bo')
plt.title('Распределение вероятностей продления платной подписки после активации промокода')
plt.xlabel('Количество продлений')
plt.ylabel('Плотность вероятности')
plt.show()

**Вывод:** При отправке 1164 промокодов вероятность получить менее 100 продлений около 0.05 *(вероятность выполнить план - 100 продлений примерно 95%, как в условии)*

### Оценка вероятности открытия 399.5 тысяч уведомлений из 1 миллиона отправленных 

Отдел маркетинга рассылает клиентам push-уведомления в мобильном приложении. Клиенты могут открыть его или не открывать. Известно, что уведомления открывают около 40% получивших клиентов. Отдел планирует разослать 1 млн уведомлений. С помощью аппроксимации построим примерный график распределения

In [51]:
success_prob = 0.4
n = 1000000
goal_amount = 399500

mu = n * success_prob
sigma = np.sqrt(n*success_prob*(1-success_prob))

x = list(range(max(0, int(mu - 4 * sigma)), int(mu + 4 * sigma)))

graph = plt.plot(x, st.binom.pmf(x, n, success_prob))
plt.title('Распределение вероятностей открытия уведомления')
plt.xlabel('Количество уведомлений')
plt.ylabel('Вероятность')
plt.show()

А теперь оценим вероятность левого хвоста от 399.5 с помощью функции `cdf()`

In [52]:
openning_distr = st.norm(mu, sigma)
print(f'Вероятность, что откроют не более 399.5 тысяч писем: {openning_distr.cdf(399500)}')

**Вывод:** Вероятность, что письма откроют не более 399.5 тысяч пользователей составляет **0.15**

**Общий вывод по Распределениям:**
- Чтобы вероятность выполнить план по 100 продлениям после акции с отправкой промокодов на месяц бесплатной подписки составляла 95%, необходимо отправить **1164 промокода**
- Вероятность, что из 1 миллиона отправленных в мобильном приложении уведомлений, будут открыты не более 399.5 тысяч, составляет **0.15**

## Общий вывод

### Предобработка данных
- Проведена проверка на отсутствие пропусков в данных
- Столбец `'date'` датафрейма `rides_df` приведен к формату datetime
- Устранены неявные дубликаты в столбце `'name'` датафрейма `users_df`

### Исследовательский анализ данных
- Из данных исключены поездки с аномально маленькой длительностью и большим расстоянием (0.5 минут и 4000 метров)
- Проанализирована частота встречаемости городов среди пользователей. Наибольшее количество пользователей - из **Пятигорска, их 219**. Далее следует **Екатеринбург с показателем 204** пользователя. Наименьшее количество пользователей - из **Москвы, их 168**.
- Проанализировано количество пользоваталей с подпиской и без, а также их доля среди общего количества пользоваталей: **835 пользователей (54%) не пользуются платной подпиской**, а **699 (46%) пользуются**.
- Проанализированы данные о возрасте пользователей: Распредедение возрастов пользователей близко к нормальному распределению **со средним 26 лет**, стандартным отклонением 8.7. Убедились, что в данных о возрасте отсутствуют аномалии.
- Исследованы данные о расстоянии, которое проезжают пользователи во время поездки. Пользователи сервиса в среднем проезжают **около 3100 метров**, это значение близко и к **медиане (3133 метра)** и к **среднему значению по выборке (3070 метров)**.
- Исследованы данные о длительности поездок. Распредедение длительности поездок близко к нормальному распределению со **средним 17 минут**, **стандартным отклонением почти 6 минут**. Медиана составляет также 17 минут. Устранена аномалия (ряд поездок с расстоянием 4 километра и более, и временем 0.5 минуты).

### Объединение данных и визуализация
- Создан новый датафрейм, который содержит объединенные данные о пользователях, поездках и подписках.
- Создано два датафрейма, которые содержат полную информацию о всех поездках пользователей с подпиской и без подписки
- Проведена визуализация распределений длительности и расстояния в разрезе поездок с подпиской и без
- Проведено сравнение длительности и расстояния в разрезе поездок с подпиской и без: 
    - Поездок без подписок почти в 2 раза больше (6500 с подпиской и 11473 без)
    - **Среднее расстояние** у поездок с подписками немного выше *(3115 против 3028 метров)*, это может быть связано с тем, что среди поездок без подписки есть некоторое количество поездок с расстоянием **менее 500** метров. Медианы почти не отличаются. В 75% поездок с подписками было пройдено **3560 метров** или меньше, а в 75% поездок без подписки расстояние было **3883**, это значительная разница
    - **Средняя длительность** поездок с подписками почти на 1 минуту больше, чем без подписок *(18.5 против 17.5)*, медианная на 0.7 минуты больше *(18.1 против 17.4)*. 75% поездок с подпиской завершены за 22 минуты или меньше, а без подписки - 21.6 минут или меньше, разница не так значительна.

### Проверка статистических гипотез
- Так как статистически доказано, что есть достаточные основания отклонить гипотезу о равенстве средних расстояний в пользу альтернативной правосторонней гипотезы, что среднее расстояние у пользователей с подпиской больше, чем у пользователей без подписки, **пользователи с подпиской выгоднее, чем без подписки**. Рекомендация: чтобы увеличить выручку и вырасти как бизнес, стоит увеличить количество пользоваталей с подпиской. Например, при помощи маркетинговой акции.
- Так как у нас недостаточно оснований отвергнуть гипотезу о равенстве среднего расстояния в поездке оптимальному значению 3130 метров, но достаточно оснований отклонить ее в пользу левосторонней гипотезы при значении 3140 метров, можно заключить, что **износ самокатов не минимален**.
- Так как статистически доказано, что есть основания отвергнуть гипотезу о равенстве выручки в пользу правосторонней альтернативной гипотезы (средняя помесячная выручка пользователей с подпиской выше, чем без подписки), **пользователи с подпиской будут приносить больше денег**.

### Распределения
- Чтобы вероятность выполнить план по 100 продлениям после акции отдела маркетинга с отправкой промокодов на месяц бесплатной подписки составляла 95%, необходимо отправить **1164 промокода**
- Вероятность, что из 1 миллиона отправленных в мобильном приложении уведомлений отделом маркетинга, будут открыты не более 399.5 тысяч, составляет **0.15 или 15%**